In [ ]:
!pip3 install --upgrade pip
!pip3 install --upgrade google-cloud-vision
!pip3 install --upgrade google-cloud-storage

In [ ]:
import re
import time

In [ ]:
# Ref: https://cloud.google.com/vision/docs/pdf

def async_detect_document(gcs_source_uri, gcs_destination_uri):
    """OCR with PDF/TIFF as source files on GCS"""
    from google.cloud import vision
    from google.cloud import storage
    from google.protobuf import json_format
    # Supported mime_types are: 'application/pdf' and 'image/tiff'
    mime_type = 'application/pdf'

    # How many pages should be grouped into each json output file.
    batch_size = 2

    client = vision.ImageAnnotatorClient()
    
    feature = vision.types.Feature(
        type=vision.enums.Feature.Type.DOCUMENT_TEXT_DETECTION)

    gcs_source = vision.types.GcsSource(uri=gcs_source_uri)
    input_config = vision.types.InputConfig(
        gcs_source=gcs_source, mime_type=mime_type)

    gcs_destination = vision.types.GcsDestination(uri=gcs_destination_uri)
    output_config = vision.types.OutputConfig(
        gcs_destination=gcs_destination, batch_size=batch_size)

    async_request = vision.types.AsyncAnnotateFileRequest(
        features=[feature], input_config=input_config,
        output_config=output_config)

    operation = client.async_batch_annotate_files(
        requests=[async_request])

    print('Waiting for the operation to finish.')
    operation.result(timeout=180)

    # Once the request has completed and the output has been
    # written to GCS, we can list all the output files.
    storage_client = storage.Client()

    match = re.match(r'gs://([^/]+)/(.+)', gcs_destination_uri)
    bucket_name = match.group(1)
    prefix = match.group(2)
    
    bucket = storage_client.get_bucket(bucket_name=bucket_name)

    # List objects with the given prefix.
    blob_list = list(bucket.list_blobs(prefix=prefix))
    print('Output files:')
    for blob in blob_list:
        print(blob.name)

In [ ]:
import datetime
datetime_object_now = datetime.datetime.now()
datetime_now = datetime_object_now.strftime("%y%m%d_%H%M%S")

# EDIT HERE
gcs_source_uri = 'gs://ocr-data-source/sample_pdf_files/'
source_file = 'pdf_demo.pdf'
sink_dir = 'json_outputs_'+datetime_now+'/'
gcs_destination_uri = 'gs://ocr-data-sink/'+sink_dir

print(gcs_destination_uri)

In [ ]:
async_detect_document((gcs_source_uri+source_file), gcs_destination_uri)